In [1]:
%load_ext autoreload
%autoreload 2
from functools import partial

import keras

from datasets import example_datasets, to_numpy
from models import mixture_poissons,location_specific_linear, CustomPenalizedMixtureDecisionModel, get_mixture
from metrics import mixture_poi_loss, get_bpr_loss_func, mix_bpr, get_penalized_bpr_loss_func_mix, cross_ratio_decision, get_perturbed_bpr_func
from experiments import training_loop, training_loop_score_function_trick, score_function_trick, overall_gradient_calculation
from plotting_funcs import plot_losses, plot_frontier

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

import matplotlib.pyplot as plt
import resource
import datetime

2024-05-12 19:23:32.018800: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-12 19:23:32.066179: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 19:23:32.066208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 19:23:32.067286: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-12 19:23:32.075049: I tensorflow/core/platform/cpu_feature_guar

In [2]:
seed=360
num_components=4
learning_rate = 0.05
epochs=200
outdir = '/cluster/home/kheuto01/testdir'
penalty = 5000
threshold = 0.55
K=4
do_only=True
# tracts/distributions
S=12
# history/features
H = 3
# total timepoints
T= 500
perturbed_sigma=0.1
num_score_func_samples=5
batch_size=50

In [3]:
train_dataset, val_dataset, test_dataset = example_datasets(H, T, seed=seed, batch_size=batch_size)
train_X_THS, train_y_TS = to_numpy(train_dataset)
val_X_THS, val_y_TS = to_numpy(val_dataset)
input_shape = (H,S)

bpr_K = get_perturbed_bpr_func(K, sigma=perturbed_sigma)
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)


2024-05-12 19:23:53.258711: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-05-12 19:23:53.258747: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: s1cmp008.pax.tufts.edu
2024-05-12 19:23:53.258754: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: s1cmp008.pax.tufts.edu
2024-05-12 19:23:53.258839: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-05-12 19:23:53.258864: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-05-12 19:23:53.258869: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [4]:
model  = get_mixture(location_specific_linear, input_shape, num_components=num_components)

In [5]:
def do_it_all(mix, num_score_func_samples, y_BS, threshold, penalty, bpr_K):

    # add constant to avoid log 0
    print('before samp')
    sample_y_MBS = mix.sample(num_score_func_samples)#+1e-13
    print('got samp')

    sample_log_probs_MBS = mix.log_prob(sample_y_MBS)
    print('got prob')
    sample_decisions_MBS = cross_ratio_decision(sample_y_MBS)
    print('got dec')
    expected_decisions_BS = tf.reduce_mean(sample_decisions_MBS, axis=0)
    print('mean dec')
    bpr_B = bpr_K(y_BS, expected_decisions_BS)
    print('got bpr')
    observed_log_prob_BS = mix.log_prob(y_BS)
    print ('got log prob obs')


    loss_B = -tf.reduce_sum(observed_log_prob_BS, axis=-1)
    print('reduced log prob')
    violate_threshold_flag_B = tf.cast(tf.greater(threshold,
                                                bpr_B),
                                        tf.float32)
    print('made a flag')
    loss_B += penalty * violate_threshold_flag_B *(threshold - bpr_B)
    print('added penalty')

    return loss_B, sample_log_probs_MBS, sample_decisions_MBS, expected_decisions_BS, observed_log_prob_BS, bpr_B

In [6]:
losses = {}
losses['train'] = {}
losses['val'] ={}
losses['train']['loss']=[]
losses['train']['nll']=[]
losses['train']['bpr']=[]
losses['val']['loss']=[]
losses['val']['nll']=[]
losses['val']['bpr']=[]
verbose=True

for epoch in range(1):
    if verbose:
        print(f'Epoch {epoch}')
    else:
        if epoch % 10 == 0:
            print(f'Epoch {epoch}')

    batch_losses = {'train': {'loss': [], 'nll': [], 'bpr': []}}
    for step, (x_BHS, y_BS) in enumerate(train_dataset):
        print(step)
        with tf.GradientTape() as jacobian_tape, tf.GradientTape() as loss_tape:
            mixture = model(x_BHS)
            print('got mix')
            loss_B, sample_log_probs_MBS, sample_decisions_MBS, expected_decisions_BS, observed_log_prob_BS, bpr_B = do_it_all(mixture, num_score_func_samples, y_BS, threshold, penalty, bpr_K)

        jacobian_pMBS = jacobian_tape.jacobian(sample_log_probs_MBS, model.trainable_weights)
        print('got jac')
        param_gradient_pBS = [score_function_trick(j, sample_decisions_MBS) for j in jacobian_pMBS]
        print('did score func trick')
        loss_gradients_BS = loss_tape.gradient(loss_B, expected_decisions_BS)
        print('got grad')
        overall_gradient = [overall_gradient_calculation(g, loss_gradients_BS) for g in param_gradient_pBS]
        print('got overall gradient')

        optimizer.apply_gradients(zip(overall_gradient, model.trainable_weights))
        print('applied')
        batch_losses['train']['loss'].append(tf.reduce_mean(loss_B))
        batch_losses['train']['nll'].append(tf.reduce_mean(tf.reduce_sum(observed_log_prob_BS, axis=-1)))
        batch_losses['train']['bpr'].append(tf.reduce_mean(bpr_B))
        print('recorded')

    losses['train']['loss'].append(tf.reduce_mean(batch_losses['train']['loss']))
    losses['train']['nll'].append(tf.reduce_mean(batch_losses['train']['nll']))
    losses['train']['bpr'].append(tf.reduce_mean(batch_losses['train']['bpr']))

    if verbose:
        # print all metrics
        print(f'Loss: {losses["train"]["loss"][-1]}')
        print(f'NLL: {losses["train"]["nll"][-1]}')
        print(f'BPR: {losses["train"]["bpr"][-1]}')

Epoch 0
0
got mix
before samp
got samp
got prob
got dec
mean dec
got bpr
got log prob obs
reduced log prob
made a flag
added penalty
got jac
did score func trick
got grad
got overall gradient
applied
recorded
1
got mix
before samp
got samp
got prob
got dec
mean dec
got bpr
got log prob obs
reduced log prob
made a flag
added penalty
got jac
did score func trick
got grad
got overall gradient
applied
recorded
2
got mix
before samp
got samp
got prob
got dec
mean dec
got bpr
got log prob obs
reduced log prob
made a flag
added penalty
got jac
did score func trick
got grad
got overall gradient
applied
recorded
3
got mix
before samp
got samp
got prob
got dec
mean dec
got bpr
got log prob obs
reduced log prob
made a flag
added penalty
got jac
did score func trick
got grad
got overall gradient
applied
recorded
4
got mix
before samp
got samp
got prob
got dec
mean dec
got bpr
got log prob obs
reduced log prob
made a flag
added penalty
got jac
did score func trick
got grad
got overall gradient
appl

In [7]:
mixture.reparameterization_type

<Reparameterization Type: NOT_REPARAMETERIZED>

In [16]:
# test if I can take gradient of samples from mixure with a tape
with tf.GradientTape() as tape:
    sample = mixture.sample(10)
    log_prob = mixture.log_prob(sample)
    print(tf.reduce_mean(log_prob))
print(tape.gradient(log_prob, model.trainable_weights))

tf.Tensor(-2.1269157, shape=(), dtype=float32)
[None, None, None, None, None, None, None, None, None]


In [17]:
this_poi = tfp.distributions.Poisson(rate=1.0)

In [18]:
this_poi.reparameterization_type

<Reparameterization Type: NOT_REPARAMETERIZED>